In [2]:
from matplotlib import style
import matplotlib.pyplot as plt 

In [46]:
import numpy as np
import pandas as pd
import json

In [4]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
fires_df = pd.read_csv('../Fires.csv')

In [6]:
fires_clean = fires_df[fires_df["STATE"] == "TX"]
fires_clean.head()

,Unnamed: 0,STATE,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
283,283,TX,2005,3.0,B,Arson,33.786111,-96.150000
284,284,TX,2005,55.0,C,Debris Burning,31.312500,-94.270833
285,285,TX,2005,2.0,B,Campfire,33.345833,-97.595833
286,286,TX,2005,0.3,B,Smoking,31.386667,-95.150556
287,287,TX,2005,55.0,C,Lightning,31.354167,-95.085278


In [7]:
fires_clean['STAT_CAUSE_DESCR'].unique()

array(['Arson', 'Debris Burning', 'Campfire', 'Smoking', 'Lightning',
       'Equipment Use', 'Miscellaneous', 'Children', 'Railroad',
       'Fireworks', 'Powerline', 'Missing/Undefined', 'Structure'],
      dtype=object)

In [8]:
list(fires_clean['STAT_CAUSE_DESCR'].unique())

['Arson',
 'Debris Burning',
 'Campfire',
 'Smoking',
 'Lightning',
 'Equipment Use',
 'Miscellaneous',
 'Children',
 'Railroad',
 'Fireworks',
 'Powerline',
 'Missing/Undefined',
 'Structure']

In [26]:
# base dictionairy 
df_fire_causes = pd.DataFrame(fires_clean['STAT_CAUSE_DESCR'].unique(), columns=['cause'])
for cur_year in range(1992, 2016):
    col_name = str(cur_year)
    df_fire_causes[col_name] = 0
df_fire_causes.set_index("cause", inplace=True)    
df_fire_causes    

,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
cause,,,,,,,,,,,,,,,,,,,,,
Arson,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Debris Burning,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Campfire,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Smoking,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lightning,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Equipment Use,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Miscellaneous,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Children,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Railroad,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
for _, value in fires_clean.iterrows():
    df_fire_causes.loc[value.STAT_CAUSE_DESCR, str(value.FIRE_YEAR)] += 1
df_fire_causes    

dict_causes[cause][year] += 1

,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
cause,,,,,,,,,,,,,,,,,,,,,
Arson,360,410,170,483,739,142,392,361,541,109,...,780,209,561,499,398,1004,260,278,231,216
Debris Burning,579,641,486,816,1255,401,578,845,993,375,...,5429,2560,6635,5638,3979,5321,3128,4774,3643,3381
Campfire,24,26,25,29,49,19,31,43,36,7,...,152,78,182,173,125,190,73,104,112,93
Smoking,22,38,16,24,70,18,67,49,63,12,...,564,121,532,409,259,559,175,214,214,197
Lightning,34,52,44,47,129,61,232,72,246,52,...,878,143,691,579,429,1495,462,282,243,200
Equipment Use,46,63,34,57,103,32,133,93,140,59,...,1904,662,2476,1706,1239,2710,849,1002,917,1019
Miscellaneous,73,167,81,149,368,71,514,261,506,114,...,4856,1578,6643,4065,2242,5989,1929,2734,2507,2587
Children,16,22,18,14,51,5,38,18,37,6,...,141,65,212,175,81,220,64,70,57,65
Railroad,9,2,18,0,1,0,32,20,28,11,...,222,38,90,105,89,160,62,61,62,29


In [11]:
fires_clean.to_csv('fires_clean2.csv', index=False)

In [35]:
df_fire_causes_json = df_fire_causes.copy() 
# df_fire_causes_json.reset_index(inplace=True)
df_fire_causes_json.head()

,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
cause,,,,,,,,,,,,,,,,,,,,,
Arson,360,410,170,483,739,142,392,361,541,109,...,780,209,561,499,398,1004,260,278,231,216
Debris Burning,579,641,486,816,1255,401,578,845,993,375,...,5429,2560,6635,5638,3979,5321,3128,4774,3643,3381
Campfire,24,26,25,29,49,19,31,43,36,7,...,152,78,182,173,125,190,73,104,112,93
Smoking,22,38,16,24,70,18,67,49,63,12,...,564,121,532,409,259,559,175,214,214,197
Lightning,34,52,44,47,129,61,232,72,246,52,...,878,143,691,579,429,1495,462,282,243,200


In [45]:

list(df_fire_causes_json.index)

['Arson',
 'Debris Burning',
 'Campfire',
 'Smoking',
 'Lightning',
 'Equipment Use',
 'Miscellaneous',
 'Children',
 'Railroad',
 'Fireworks',
 'Powerline',
 'Missing/Undefined',
 'Structure']

In [51]:
cause_count_lists = json.loads(df_fire_causes_json.to_json(orient='values'))
cause_count_lists[1]

causes_count_dict = {}
for key, cur_cause in enumerate(list(df_fire_causes_json.index)):
    causes_count_dict[cur_cause] = cause_count_lists[key]

json_data = json.dumps(causes_count_dict)   
with open("cause_count.json", "w") as fh:
    fh.write(json_data)
print("All Done")    


All Done
